<a href="https://colab.research.google.com/github/C4st3ll4n/google_colab/blob/master/padronizacao_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
base = pd.read_csv('census.csv')

In [0]:
base.head()

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [0]:
base.income.unique()

array([' <=50K', ' >50K'], dtype=object)

In [0]:
def converte_classe(rotulo):
    if rotulo == ' >50K':
        return 1
    else:
     return 0

In [0]:
base.income = base.income.apply(converte_classe)

In [0]:
base.income

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        1
8        1
9        1
10       1
11       1
12       0
13       0
14       1
15       0
16       0
17       0
18       0
19       1
20       1
21       0
22       0
23       0
24       0
25       1
26       0
27       1
28       0
29       0
        ..
32531    0
32532    1
32533    1
32534    0
32535    0
32536    1
32537    0
32538    1
32539    1
32540    0
32541    0
32542    0
32543    0
32544    0
32545    1
32546    0
32547    0
32548    0
32549    0
32550    0
32551    0
32552    0
32553    0
32554    1
32555    0
32556    0
32557    1
32558    0
32559    0
32560    1
Name: income, Length: 32561, dtype: int64

In [0]:
X = base.drop('income', axis=1)
Y = base.income

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_treinamento, X_teste, Y_treinamento, Y_teste = train_test_split(X, Y, test_size=0.2)

In [0]:
import tensorflow as tf

In [0]:
base.columns

Index(['age', 'workclass', 'final-weight', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loos', 'hour-per-week', 'native-country',
       'income'],
      dtype='object')

In [0]:
workclass = tf.feature_column.categorical_column_with_hash_bucket('workclass', hash_bucket_size=100 )
education = tf.feature_column.categorical_column_with_hash_bucket('education', hash_bucket_size=100 )
marital_status = tf.feature_column.categorical_column_with_hash_bucket('marital-status', hash_bucket_size=100 )
occupation = tf.feature_column.categorical_column_with_hash_bucket('occupation', hash_bucket_size=100 )
relationship = tf.feature_column.categorical_column_with_hash_bucket('relationship', hash_bucket_size=100 )
race = tf.feature_column.categorical_column_with_hash_bucket('race', hash_bucket_size=100 )
country = tf.feature_column.categorical_column_with_hash_bucket('native-country', hash_bucket_size=100 )

In [0]:
sex = tf.feature_column.categorical_column_with_vocabulary_list('sex',vocabulary_list=[' Male', ' Female'])

In [0]:
def padroniza_age(idade):
    return tf.divide(tf.subtract(tf.cast(idade, tf.float32), tf.constant(38.58)), tf.constant(13.64))

def padroniza_finalW(weight):
    return tf.divide(tf.subtract(tf.cast(weight, tf.float32), tf.constant(189778.36)), tf.constant(105549.997))

def padroniza_education(edu):
    return tf.divide(tf.subtract(tf.cast(edu, tf.float32), tf.constant(10.08)), tf.constant(2.57))

def padroniza_capitalgain(cgn):
    return tf.divide(tf.subtract(tf.cast(cgn, tf.float32), tf.constant(1077.64)), tf.constant(7385.29))

def padroniza_capitalloos(cloos):
    return tf.divide(tf.subtract(tf.cast(cloos, tf.float32), tf.constant(87.30)), tf.constant(402.96))

def padroniza_hour(hora):
    return tf.divide(tf.subtract(tf.cast(hora, tf.float32), tf.constant(40.43)), tf.constant(12.34))

In [0]:
age = tf.feature_column.numeric_column('age', normalizer_fn=padroniza_age)
hour = tf.feature_column.numeric_column('hour-per-week', normalizer_fn=padroniza_hour)
final_weight = tf.feature_column.numeric_column('final-weight', normalizer_fn=padroniza_finalW)
education_num = tf.feature_column.numeric_column('education-num', normalizer_fn=padroniza_education)
capital_gain = tf.feature_column.numeric_column('capital-gain', normalizer_fn=padroniza_capitalgain)
capital_loos = tf.feature_column.numeric_column('capital-loos', normalizer_fn=padroniza_capitalloos)

In [0]:
colunas = [age, workclass, final_weight, education, education_num,
       marital_status, occupation, relationship, race, sex,
       capital_gain, capital_loos, hour, country]

In [0]:
funcao_treinamento = tf.estimator.inputs.pandas_input_fn(X_treinamento, Y_treinamento, 180, None, True)

In [0]:
classificador = tf.estimator.DNNClassifier([8, 8, 8], colunas)

W0831 19:41:58.585247 140124169480064 estimator.py:1811] Using temporary folder as model directory: /tmp/tmpfnm1nlbl


In [0]:
embedded_wc = tf.feature_column.embedding_column(workclass,len(base.workclass.unique()))
embedded_edu = tf.feature_column.embedding_column(education,len(base.education.unique()))
embedded_mtl = tf.feature_column.embedding_column(marital_status,len(base['marital-status'].unique()))
embedded_occp = tf.feature_column.embedding_column(occupation,len(base.occupation.unique()))
embedded_rltsp = tf.feature_column.embedding_column(relationship,len(base.relationship.unique()))
embedded_race = tf.feature_column.embedding_column(race,len(base.race.unique()))
embedded_sex = tf.feature_column.embedding_column(sex,len(base.sex.unique()))
embedded_cnt = tf.feature_column.embedding_column(country,len(base['native-country'].unique()))

In [0]:
coluna_embedded = [age, embedded_wc, final_weight, embedded_edu, education_num,
       embedded_mtl, embedded_occp, embedded_rltsp, embedded_race, embedded_sex,
       capital_gain, capital_loos, hour, embedded_cnt]

In [0]:
classificador = tf.estimator.DNNClassifier([8, 8, 8], coluna_embedded)

W0831 19:41:58.652577 140124169480064 estimator.py:1811] Using temporary folder as model directory: /tmp/tmpiy6vkah6


In [0]:
classificador.train(funcao_treinamento, steps=10000)

W0831 19:42:03.346221 140124169480064 basic_session_run_hooks.py:724] It seems that global step (tf.train.get_global_step) has not been increased. Current value (could be stable): 39 vs previous value: 39. You could increase the global step by passing tf.train.get_global_step() to Optimizer.apply_gradients or Optimizer.minimize.
W0831 19:42:03.865481 140124169480064 basic_session_run_hooks.py:724] It seems that global step (tf.train.get_global_step) has not been increased. Current value (could be stable): 101 vs previous value: 101. You could increase the global step by passing tf.train.get_global_step() to Optimizer.apply_gradients or Optimizer.minimize.
W0831 19:42:04.683331 140124169480064 basic_session_run_hooks.py:724] It seems that global step (tf.train.get_global_step) has not been increased. Current value (could be stable): 201 vs previous value: 201. You could increase the global step by passing tf.train.get_global_step() to Optimizer.apply_gradients or Optimizer.minimize.
W08

In [0]:
funcao_teste = tf.estimator.inputs.pandas_input_fn(X_teste, Y_teste, 180, shuffle=False)

In [0]:
classificador.evaluate(funcao_teste)

W0831 19:43:23.850471 140124169480064 metrics_impl.py:804] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W0831 19:43:23.879134 140124169480064 metrics_impl.py:804] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.


{'accuracy': 0.852756,
 'accuracy_baseline': 0.7649317,
 'auc': 0.9068259,
 'auc_precision_recall': 0.7649845,
 'average_loss': 0.31632027,
 'global_step': 10000,
 'label/mean': 0.23506832,
 'loss': 55.680916,
 'precision': 0.7259084,
 'prediction/mean': 0.23318638,
 'recall': 0.6002613}

In [0]:
def padroniza_age(idade):
    return tf.divide(tf.subtract(tf.cast(idade, tf.float32), tf.constant(38.58)), tf.constant(13.64))

def padroniza_finalW(weight):
    return tf.divide(tf.subtract(tf.cast(weight, tf.float32), tf.constant(189778.36)), tf.constant(105549.997))

def padroniza_education(edu):
    return tf.divide(tf.subtract(tf.cast(edu, tf.float32), tf.constant(10.08)), tf.constant(2.57))

def padroniza_capitalgain(cgn):
    return tf.divide(tf.subtract(tf.cast(cgn, tf.float32), tf.constant(1077.64)), tf.constant(7385.29))

def padroniza_capitalloos(cloos):
    return tf.divide(tf.subtract(tf.cast(cloos, tf.float32), tf.constant(87.30)), tf.constant(402.96))

def padroniza_hour(hora):
    return tf.divide(tf.subtract(tf.cast(hora, tf.float32), tf.constant(40.43)), tf.constant(12.34))